# Hola, mundo en LangChain

## Instalar librerías principales y configuración de API Key de OpenAI

In [1]:
from getpass import getpass
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Carga de documents

In [2]:
import requests
from langchain.document_loaders import PyPDFLoader

urls = [
    'https://arxiv.org/pdf/2306.06031v1.pdf',
    'https://arxiv.org/pdf/2306.12156v1.pdf',
    'https://arxiv.org/pdf/2306.14289v1.pdf',
    'https://arxiv.org/pdf/2305.10973v1.pdf',
    'https://arxiv.org/pdf/2306.13643v1.pdf'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper_{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()

Descargado paper_1.pdf
Descargado paper_2.pdf
Descargado paper_3.pdf
Descargado paper_4.pdf
Descargado paper_5.pdf
Contenido de ml_papers:



In [3]:
type(ml_papers), len(ml_papers), ml_papers[3]

(list,
 57,
 Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-12T00:32:18+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-12T00:32:18+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'templateversion': 'IJCAI.2023.0', 'title': '', 'trapped': '/False', 'source': 'paper_1.pdf', 'total_pages': 7, 'page': 3, 'page_label': '4'}, page_content='Figure 1: FinGPT Framework.\n4.1 Data Sources\nThe first stage of the FinGPT pipeline involves the collec-\ntion of extensive financial data from a wide array of online\nsources. These include, but are not limited to:\n• Financial news: Websites such as Reuters, CNBC, Yahoo\nFinance, among others, are rich sources of financial news\nand market updates. These sites provide valuable informa-\ntion on market trends, company earnings, macroeconomic\nindicators, and other financial events.\n• Social media: P

## Split de documents

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
    )

documents = text_splitter.split_documents(ml_papers)

In [5]:
len(documents), documents[10]

(211,
 Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-12T00:32:18+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-12T00:32:18+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'templateversion': 'IJCAI.2023.0', 'title': '', 'trapped': '/False', 'source': 'paper_1.pdf', 'total_pages': 7, 'page': 2, 'page_label': '3'}, page_content='highly volatile, changing rapidly in response to news events\nor market movements.\nTrends, often observable through websites like Seeking\nAlpha, Google Trends, and other finance-oriented blogs and\nforums, offer critical insights into market movements and in-\nvestment strategies. They feature:\n• Analyst perspectives: These platforms provide access to\nmarket predictions and investment advice from seasoned\nfinancial analysts and experts.\n• Market sentiment: The discourse on these platforms can\nreflect t

## Embeddings e ingesta a base de datos vectorial

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
    )

## Modelos de chat y cadenas para consulta de información

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

/tmp/ipykernel_18653/474697823.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [9]:
query = "qué es fingpt?"
qa_chain.invoke(query)

{'query': 'qué es fingpt?',
 'result': 'FinGPT es un modelo de lenguaje de código abierto diseñado para el procesamiento de lenguaje natural en el ámbito financiero. Adopta un enfoque centrado en los datos y cuenta con un marco de trabajo de extremo a extremo con cuatro capas. Su objetivo es estimular la innovación, democratizar los modelos de lenguaje financiero y desbloquear nuevas oportunidades en finanzas abiertas. Además, ofrece tutoriales prácticos y demostraciones de aplicaciones financieras para tareas como servicios de asesoramiento robótico, trading cuantitativo y desarrollo de bajo código.'}

In [10]:
query = "qué hace complicado entrenar un modelo como el fingpt?"
qa_chain.invoke(query)

{'query': 'qué hace complicado entrenar un modelo como el fingpt?',
 'result': 'Entrenar un modelo como FinGPT puede ser complicado debido a la intensiva exigencia computacional que conlleva. Por ejemplo, BloombergGPT utilizó aproximadamente 1.3 millones de horas de GPU para su entrenamiento, lo que, al calcularse con la tarifa de $2.3 de la nube de AWS, se traduce en un costo impresionante de alrededor de $3 millones por entrenamiento. En contraste con el alto costo computacional de modelos como BloombergGPT, FinGPT presenta una solución más accesible al enfocarse en la adaptación ligera de los principales LLM de código abierto. El costo de adaptación disminuye significativamente, estimado en menos de $300 por entrenamiento.'}

In [11]:
query = "qué es fast segment?"
qa_chain.invoke(query)

{'query': 'qué es fast segment?',
 'result': 'Fast Segment, también conocido como FastSAM, es un modelo ligero diseñado para aplicaciones móviles que se basa en el modelo Segment Anything (SAM). FastSAM reemplaza el codificador de imágenes pesado de SAM con uno más ligero para hacerlo más adecuado para dispositivos móviles con recursos limitados. A través de un proceso de destilación desacoplada, FastSAM logra un rendimiento satisfactorio al segmentar objetos de interés en imágenes, incluso con fuentes de entrenamiento limitadas. En comparación con SAM, FastSAM muestra una mejora en la velocidad de ejecución y resultados satisfactorios en tareas como detección de bordes y segmentación de objetos salientes.'}

In [12]:
query = "cuál es la diferencia entre fast sam y mobile sam?"
qa_chain.invoke(query)

{'query': 'cuál es la diferencia entre fast sam y mobile sam?',
 'result': 'La diferencia principal entre FastSAM y MobileSAM es que MobileSAM es más rápido y más pequeño en tamaño en comparación con FastSAM. MobileSAM tiene menos parámetros (10M frente a 68M) y es cuatro veces más rápido en el procesamiento de imágenes (10ms frente a 40ms). Además, MobileSAM tiene un rendimiento superior en términos de mIoU (0.73 en comparación con 0.41 de FastSAM), lo que sugiere que la predicción de máscaras de MobileSAM es más cercana a la del SAM original que la de FastSAM.'}